Installation of Libraries

In [2]:
print("Hello world")

Hello world


In [2]:
!pip install torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/6d/13/b5e8bacd980b2195f8a1741ce11cbb9146568607795d5e4ff510dcff1064/torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl.metadata
  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Obtaining dependency information for nvidia-cudnn-cu12==8.9.2.26 from https://files.pythonhosted.org/packages/ff/74/a2e2be7fb83aaedec84f391f082cf765dfb635e7caa9b49065f73e4835d8/nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_c

In [3]:
!pip install telebot

  Using cached telebot-0.0.5-py3-none-any.whl (4.8 kB)
  Using cached pyTelegramBotAPI-4.14.0-py3-none-any.whl

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!python -m pip install --upgrade pip
!pip install torch

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/47/6a/453160888fab7c6a432a6e25f8afe6256d0d9f2cbd25971021da6491d899/pip-23.3.1-py3-none-any.whl.metadata
  Using cached pip-23.3.1-py3-none-any.whl.metadata (3.5 kB)
Using cached pip-23.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11

In [5]:
!pip install pickle-mixin

  Using cached pickle_mixin-1.0.2-py3-none-any.whl


Actual Implementation

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import datetime 

import telebot
from telebot import types 

In [7]:
hidden_size = 128
num_layers = 2

class ModelNN(nn.Module):
    def __init__(self):
        super(ModelNN, self).__init__()
        self.l1 = nn.Linear(6, 128)
        self.dropout = nn.Dropout(0.1)
        self.l2 = nn.Linear(128, 64)
        self.l3 = nn.Linear(64, 5) # 5 classes: from 1-5

    def forward(self, x):
        x = self.l1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.l2(x)
        x = F.relu(x)
        x = self.dropout(x)
        output = self.l3(x)
        return output

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size = 6, hidden_size = hidden_size, num_layers = num_layers, batch_first= True)
        self.output = nn.Linear(hidden_size, 6)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.output(x)
        return x

# Load LSTM
lstm = torch.load('lstm.pth')
lstm.eval()  # Put the model in evaluation model

# Load Neural Network 
classifier = torch.load('classify_nn.pth')
classifier.eval()  # Put the model in evaluation model 

# Load standard scaler 
with open('lstm_sc.pkl','rb') as f:
    sc = pickle.load(f)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
path = "s3://iotcore-to-s3-bucket/data/data.csv"
df = pd.read_csv(path)
df

,day,time,temperature,humidity,gas,motion,dampness,trash,rating
0,2023-11-01,0:00,25.877322,76.955939,775.0,0,4095.000000,15.0,5.00
1,2023-11-01,0:05,25.139135,76.425273,725.0,0,4095.000000,15.0,5.00
2,2023-11-01,0:10,26.843238,77.102513,769.0,0,4095.000000,15.0,5.00
3,2023-11-01,0:15,25.344632,76.212004,724.0,0,4095.000000,15.0,5.00
4,2023-11-01,0:20,26.764051,76.274117,745.0,0,4095.000000,15.0,4.00
...,...,...,...,...,...,...,...,...,...
12094,2023-12-12,23:50,25.228414,78.670817,1123.0,0,2560.914603,3.0,4.00
12095,2023-12-12,23:55,26.209149,78.045677,1284.0,0,2527.698203,3.0,3.00
12096,2023-11-13,22:40,30.700001,60.000000,1504.0,10,4095.000000,15.0,4.95
12097,2023-11-13,22:40,30.700001,60.000000,1478.0,10,4095.000000,15.0,4.95


In [9]:
curr_df = pd.read_csv(path)
curr_df = curr_df.iloc[-24:, 2:-1] # take last 24 datapoints and remove ratings, date and time
curr_df

,temperature,humidity,gas,motion,dampness,trash
12075,25.352686,81.812249,1303.0,10,2539.400689,3.0
12076,25.286756,80.563007,1396.0,3,2569.288370,3.0
12077,25.983841,80.523062,1445.0,8,2574.292953,3.0
12078,25.813435,81.497120,1348.0,7,2570.832162,3.0
12079,26.150245,81.782440,1176.0,0,2554.995460,3.0
12080,25.309083,78.560073,1192.0,0,2551.334335,3.0
12081,25.911777,79.555061,1134.0,0,2560.659132,3.0
12082,26.179877,78.652718,1133.0,0,2570.963033,3.0
12083,25.292050,79.030021,1221.0,0,2544.486094,3.0
12084,25.944225,79.344535,1122.0,0,2583.068353,3.0


In [13]:
# loading telegram bot 
BOT_TOKEN = '6528183755:AAFdxN_1-wixTemIpCBq8j-ulJHTI_hAgZ0'

bot = telebot.TeleBot(BOT_TOKEN)

markup = types.InlineKeyboardMarkup(row_width=1)

@bot.message_handler(commands=['start'])
def pick_role(message): 
    cleaner = types.InlineKeyboardButton('cleaner', callback_data='cleaner')
    owner = types.InlineKeyboardButton('building owner', callback_data='owner')
    
    markup = types.InlineKeyboardMarkup(row_width=1)
    markup.add(cleaner, owner)
    bot.send_message(message.chat.id, 'pick a role!', reply_markup=markup)

        
@bot.callback_query_handler(func=lambda call: call.data == 'cleaner')
def cleaner_select(callback): 
    pred_clean = types.InlineKeyboardButton('predict next cleaning', callback_data='pred_clean')
    temp_trend = types.InlineKeyboardButton('check temperature trend', callback_data='temp_trend')
    hum_trend = types.InlineKeyboardButton('check humidity trend', callback_data='hum_trend')
    gas_trend = types.InlineKeyboardButton('check gas trend', callback_data='gas_trend')
    motion_trend = types.InlineKeyboardButton('check motion trend', callback_data='motion_trend')
    damp_trend = types.InlineKeyboardButton('check dampness trend', callback_data='damp_trend')
    trash_trend = types.InlineKeyboardButton('check trash trend', callback_data='trash_trend')
    rating_trend = types.InlineKeyboardButton('check rating trend', callback_data='rating_trend')
    
    markup = types.InlineKeyboardMarkup(row_width=1)
    markup.add(pred_clean, temp_trend, hum_trend, gas_trend, motion_trend, damp_trend, trash_trend, rating_trend)
    bot.send_message(callback.message.chat.id, 'what do you wish to check?', reply_markup=markup)

@bot.callback_query_handler(func=lambda call: call.data == 'owner')
def owner_select(callback): 
    temp_stats = types.InlineKeyboardButton('check temperature stats', callback_data='temp_stats')
    hum_stats = types.InlineKeyboardButton('check humidity stats', callback_data='hum_stats')
    gas_stats = types.InlineKeyboardButton('check gas stats', callback_data='gas_stats')
    motion_stats = types.InlineKeyboardButton('check motion stats', callback_data='motion_stats')
    damp_stats = types.InlineKeyboardButton('check dampness stats', callback_data='damp_stats')
    trash_stats = types.InlineKeyboardButton('check trash stats', callback_data='trash_stats')
    rating_stats = types.InlineKeyboardButton('check rating stats', callback_data='rating_stats')
    
    markup = types.InlineKeyboardMarkup(row_width=1)
    markup.add(temp_stats, hum_stats, gas_stats, motion_stats, damp_stats, trash_stats, rating_stats)
    bot.send_message(callback.message.chat.id, 'what do you wish to check?', reply_markup=markup)
    
@bot.callback_query_handler(func=lambda call: call.data == 'pred_clean')
def pred_clean(callback):
    markup = types.InlineKeyboardMarkup(row_width=1)
    
    curr_df = pd.read_csv(path)
    curr_df = curr_df.iloc[-24:, 2:-1] # take last 24 datapoints and remove ratings, date and time
    
    test_data = curr_df.values.tolist()
    test_data = sc.transform(test_data)
    test_data = torch.tensor(np.array(test_data)).float()
    
    count = 0
    with torch.no_grad():
        while True:
            curr = lstm(test_data)
            test_data = curr
            
            outputs = classifier(torch.Tensor(np.array([np.array(curr[-1])])))
            _, predicted = torch.max(outputs.data, 1)
            if (np.array(predicted) < 3): # need to clean (if rating 0, 1, 2-> 1, 2, 3)
                break 
                
            count += 1
    
    msg = "Cleaning is predicted to be required in: " + str(count*5) + " minutes."
    bot.send_message(callback.message.chat.id, msg, reply_markup=markup)
    
@bot.callback_query_handler(func=lambda call: call.data == 'temp_trend')
def temp_trend(callback):
    curr_df = pd.read_csv(path)
    curr_df = curr_df.iloc[-24:, 2:-1] # take last 24 datapoints and remove ratings, date and time 
    
    arr = curr_df['temperature'].values.tolist()
    
    test_data = curr_df.values.tolist()
    test_data = sc.transform(test_data)
    test_data = torch.tensor(np.array(test_data)).float()
    
    with torch.no_grad():
        for i in range(24): 
            curr = lstm(test_data)
            test_data = curr
            
            curr = sc.inverse_transform(curr)
            arr.append(curr[-1, 0]) # make sure temperature is column 0 
            
    plt.figure()
    plt.plot(np.linspace(-120, 120, 48), arr, c="r")
    plt.xlabel("Time/mins")
    plt.ylabel("Temperature/degC")
    plt.axvline(x=0, color='b', linestyle='--', label='now')
    plt.legend()
    plt.savefig("temp_plot.png")
        
    # Send the image
    with open('temp_plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo, caption='Temperature Trend')
        
@bot.callback_query_handler(func=lambda call: call.data == 'hum_trend')
def hum_trend(callback):
    curr_df = pd.read_csv(path)
    curr_df = curr_df.iloc[-24:, 2:-1] # take last 24 datapoints and remove ratings, date and time
    
    arr = curr_df['humidity'].values.tolist()
    
    test_data = curr_df.values.tolist()
    test_data = sc.transform(test_data)
    test_data = torch.tensor(np.array(test_data)).float()
    
    with torch.no_grad():
        for i in range(24): 
            curr = lstm(test_data)
            test_data = curr
            
            curr = sc.inverse_transform(curr)
            arr.append(curr[-1, 1]) 
            
    plt.figure()
    plt.plot(np.linspace(-120, 120, 48), arr, c="r")
    plt.xlabel("Time/mins")
    plt.ylabel("Humidity/%")
    plt.axvline(x=0, color='b', linestyle='--', label='now')
    plt.legend()
    plt.savefig("hum_plot.png")
        
    # Send the image
    with open('hum_plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo, caption='Humidity Trend')
        
@bot.callback_query_handler(func=lambda call: call.data == 'gas_trend')
def gas_trend(callback):
    curr_df = pd.read_csv(path)
    curr_df = curr_df.iloc[-24:, 2:-1] # take last 24 datapoints and remove ratings, date and time
    
    arr = curr_df['gas'].values.tolist()
    
    test_data = curr_df.values.tolist()
    test_data = sc.transform(test_data)
    test_data = torch.tensor(np.array(test_data)).float()
    
    with torch.no_grad():
        for i in range(24): 
            curr = lstm(test_data)
            test_data = curr
            
            curr = sc.inverse_transform(curr)
            arr.append(curr[-1, 2]) 
            
    plt.figure()
    plt.plot(np.linspace(-120, 120, 48), arr, c="r")
    plt.xlabel("Time/mins")
    plt.ylabel("Gas")
    plt.axvline(x=0, color='b', linestyle='--', label='now')
    plt.legend()
    plt.savefig("gas_plot.png")
        
    # Send the image
    with open('gas_plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo, caption='Gas Trend')
        
@bot.callback_query_handler(func=lambda call: call.data == 'motion_trend')
def motion_trend(callback):
    curr_df = pd.read_csv(path)
    curr_df = curr_df.iloc[-24:, 2:-1] # take last 24 datapoints and remove ratings, date and time 
    
    arr = curr_df['motion'].values.tolist()
    
    test_data = curr_df.values.tolist()
    test_data = sc.transform(test_data)
    test_data = torch.tensor(np.array(test_data)).float()
    
    with torch.no_grad():
        for i in range(24): 
            curr = lstm(test_data)
            test_data = curr
            
            curr = sc.inverse_transform(curr)
            arr.append(curr[-1, 3]) 
            
    plt.figure()
    plt.plot(np.linspace(-120, 120, 48), arr, c="r")
    plt.xlabel("Time/mins")
    plt.ylabel("Motion")
    plt.axvline(x=0, color='b', linestyle='--', label='now')
    plt.legend()
    plt.savefig("motion_plot.png")
        
    # Send the image
    with open('motion_plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo, caption='Motion Trend')
        
@bot.callback_query_handler(func=lambda call: call.data == 'damp_trend')
def damp_trend(callback):
    curr_df = pd.read_csv(path)
    curr_df = curr_df.iloc[-24:, 2:-1] # take last 24 datapoints and remove ratings, date and time
    
    arr = curr_df['dampness'].values.tolist()
    
    test_data = curr_df.values.tolist()
    test_data = sc.transform(test_data)
    test_data = torch.tensor(np.array(test_data)).float()
    
    with torch.no_grad():
        for i in range(24): 
            curr = lstm(test_data)
            test_data = curr
            
            curr = sc.inverse_transform(curr)
            arr.append(curr[-1, 4]) 
            
    plt.figure()
    plt.plot(np.linspace(-120, 120, 48), arr, c="r")
    plt.xlabel("Time/mins")
    plt.ylabel("Dampness")
    plt.axvline(x=0, color='b', linestyle='--', label='now')
    plt.legend()
    plt.savefig("damp_plot.png")
        
    # Send the image
    with open('damp_plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo, caption='Dampness Trend')
        
@bot.callback_query_handler(func=lambda call: call.data == 'trash_trend')
def trash_trend(callback):
    curr_df = pd.read_csv(path)
    curr_df = curr_df.iloc[-24:, 2:-1] # take last 24 datapoints and remove ratings, date and time 
    
    arr = curr_df['trash'].values.tolist()
    
    test_data = curr_df.values.tolist()
    test_data = sc.transform(test_data)
    test_data = torch.tensor(np.array(test_data)).float()
    
    with torch.no_grad():
        for i in range(24): 
            curr = lstm(test_data)
            test_data = curr
            
            curr = sc.inverse_transform(curr)
            arr.append(curr[-1, 5]) 
            
    plt.figure()
    plt.plot(np.linspace(-120, 120, 48), arr, c="r")
    plt.xlabel("Time/mins")
    plt.ylabel("Trash")
    plt.axvline(x=0, color='b', linestyle='--', label='now')
    plt.legend()
    plt.savefig("trash_plot.png")
        
    # Send the image
    with open('trash_plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo, caption='Trash Trend')
        
@bot.callback_query_handler(func=lambda call: call.data == 'rating_trend')
def rating_trend(callback):
    curr_df = pd.read_csv(path)
    
    arr = curr_df.iloc[-24:, -1].values.tolist()
    curr_df = curr_df.iloc[-24:, 2:-1] # take last 24 datapoints and remove ratings, date and time
    
    test_data = curr_df.values.tolist()
    test_data = sc.transform(test_data)
    test_data = torch.tensor(np.array(test_data)).float()
    
    with torch.no_grad():
        for i in range(24): 
            curr = lstm(test_data)
            test_data = curr
            
            outputs = classifier(torch.Tensor(np.array([np.array(curr[-1])])))
            _, predicted = torch.max(outputs.data, 1)
            arr.append(int(np.array(predicted)[0]) + 1)
            
    plt.figure()
    plt.plot(np.linspace(-120, 120, 48), arr, c="r")
    plt.xlabel("Time/mins")
    plt.ylabel("Ratings")
    plt.axvline(x=0, color='b', linestyle='--', label='now')
    plt.legend()
    plt.savefig("ratings_plot.png")
        
    # Send the image
    with open('ratings_plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo, caption='Ratings Trend')
        
@bot.callback_query_handler(func=lambda call: call.data == 'temp_stats')
def temp_stats(callback):
    # collecting november data 
    start_dt = datetime.datetime(2023, 11, 1)
    end_dt = datetime.datetime(2023, 11, 30)

    # data preparation 
    df = pd.read_csv(path)

    # filter to only contain november data 
    df['dt_day'] = df['day'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    df = df[df['dt_day'] >= start_dt]
    df = df[df['dt_day'] <= end_dt]
    df['fill_time'] = df['time'].apply(lambda x: x.zfill(5))
    
    # plot average daily temp
    dt = start_dt 
    dt_arr = []
    temp_arr = []
    while (dt <= end_dt):
        dt_arr.append(dt.strftime("%Y-%m-%d"))
        temp_arr.append(list(df[df['dt_day'] == dt]['temperature'].values))
        dt += datetime.timedelta(days=1)

    plt.figure()
    plt.boxplot(temp_arr, labels=dt_arr)
    plt.xticks(rotation=45)
    plt.xlabel('date')
    plt.ylabel('temperature/degC')
    plt.savefig("plot.png", bbox_inches='tight')
        
    # Send the image
    with open('plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo)

    # plot average temp throughout the day 
    time_arr = []
    temp_arr = []
    curr_time = datetime.datetime(year=2023, month=1, day=1, hour=0, minute=0, second=0)
    end_time = datetime.datetime(year=2023, month=1, day=1, hour=23, minute=55, second=0)
    while (curr_time <= end_time): 
        time_arr.append(curr_time.strftime("%H:%M"))
        temp_arr.append(np.mean(df[df['fill_time'] == curr_time.strftime("%H:%M")]['temperature'].values))
        curr_time += datetime.timedelta(minutes=5)

    plt.figure()
    plt.plot(time_arr, temp_arr)
    x_values = time_arr[::12] # show every 1 hour label 
    plt.xticks(x_values, rotation=45)
    plt.xlabel('time')
    plt.ylabel('average temperature/degC')
    plt.savefig("plot.png")
        
    # Send the image
    with open('plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo)

    # output overall averages 
    msg = 'Overall Average Temperature: ' + str(np.mean(df['temperature'].values))
    bot.send_message(callback.message.chat.id, msg, reply_markup=markup)

@bot.callback_query_handler(func=lambda call: call.data == 'hum_stats')
def hum_stats(callback):
    # collecting november data 
    start_dt = datetime.datetime(2023, 11, 1)
    end_dt = datetime.datetime(2023, 11, 30)

    # data preparation 
    df = pd.read_csv(path)

    # filter to only contain november data 
    df['dt_day'] = df['day'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    df = df[df['dt_day'] >= start_dt]
    df = df[df['dt_day'] <= end_dt]
    df['fill_time'] = df['time'].apply(lambda x: x.zfill(5))
    
    # plot average daily humidity
    dt = start_dt 
    dt_arr = []
    hum_arr = []
    while (dt <= end_dt):
        dt_arr.append(dt.strftime("%Y-%m-%d"))
        hum_arr.append(list(df[df['dt_day'] == dt]['humidity'].values))
        dt += datetime.timedelta(days=1)

    plt.figure()
    plt.boxplot(hum_arr, labels=dt_arr)
    plt.xticks(rotation=45)
    plt.xlabel('date')
    plt.ylabel('humidity/%')
    plt.savefig("plot.png", bbox_inches='tight')
        
    # Send the image
    with open('plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo)

    # plot average humidity throughout the day 
    time_arr = []
    hum_arr = []
    curr_time = datetime.datetime(year=2023, month=1, day=1, hour=0, minute=0, second=0)
    end_time = datetime.datetime(year=2023, month=1, day=1, hour=23, minute=55, second=0)
    while (curr_time <= end_time): 
        time_arr.append(curr_time.strftime("%H:%M"))
        hum_arr.append(np.mean(df[df['fill_time'] == curr_time.strftime("%H:%M")]['humidity'].values))
        curr_time += datetime.timedelta(minutes=5)

    plt.figure()
    plt.plot(time_arr, hum_arr)
    x_values = time_arr[::12] # show every 1 hour label 
    plt.xticks(x_values, rotation=45)
    plt.xlabel('time')
    plt.ylabel('average humidity/%')
    plt.savefig("plot.png", bbox_inches='tight')
        
    # Send the image
    with open('plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo)

    # output overall average humidity 
    msg = 'Overall Average Humidity: ' + str(np.mean(df['humidity'].values))
    bot.send_message(callback.message.chat.id, msg, reply_markup=markup)
    
@bot.callback_query_handler(func=lambda call: call.data == 'gas_stats')
def gas_stats(callback):
    # collecting november data 
    start_dt = datetime.datetime(2023, 11, 1)
    end_dt = datetime.datetime(2023, 11, 30)

    # data preparation 
    df = pd.read_csv(path)

    # filter to only contain november data 
    df['dt_day'] = df['day'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    df = df[df['dt_day'] >= start_dt]
    df = df[df['dt_day'] <= end_dt]
    df['fill_time'] = df['time'].apply(lambda x: x.zfill(5))
    
    # plot average daily gas
    dt = start_dt 
    dt_arr = []
    gas_arr = []
    while (dt <= end_dt):
        dt_arr.append(dt.strftime("%Y-%m-%d"))
        gas_arr.append(list(df[df['dt_day'] == dt]['gas'].values))
        dt += datetime.timedelta(days=1)

    plt.figure()
    plt.boxplot(gas_arr, labels=dt_arr)
    plt.xticks(rotation=45)
    plt.xlabel('date')
    plt.ylabel('gas')
    plt.savefig("plot.png", bbox_inches='tight')
        
    # Send the image
    with open('plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo)

    # plot average dampness throughout the day 
    time_arr = []
    gas_arr = []
    curr_time = datetime.datetime(year=2023, month=1, day=1, hour=0, minute=0, second=0)
    end_time = datetime.datetime(year=2023, month=1, day=1, hour=23, minute=55, second=0)
    while (curr_time <= end_time): 
        time_arr.append(curr_time.strftime("%H:%M"))
        gas_arr.append(np.mean(df[df['fill_time'] == curr_time.strftime("%H:%M")]['gas'].values))
        curr_time += datetime.timedelta(minutes=5)

    plt.figure()
    plt.plot(time_arr, gas_arr)
    x_values = time_arr[::12] # show every 1 hour label 
    plt.xticks(x_values, rotation=45)
    plt.xlabel('time')
    plt.ylabel('average gas')
    plt.savefig("plot.png", bbox_inches='tight')
        
    # Send the image
    with open('plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo)

    # output overall average gas 
    msg = 'Overall Average Gas: ' + str(np.mean(df['gas'].values))
    bot.send_message(callback.message.chat.id, msg, reply_markup=markup)
    
@bot.callback_query_handler(func=lambda call: call.data == 'motion_stats')
def motion_stats(callback):
    # collecting november data 
    start_dt = datetime.datetime(2023, 11, 1)
    end_dt = datetime.datetime(2023, 11, 30)

    # data preparation 
    df = pd.read_csv(path)

    # filter to only contain november data 
    df['dt_day'] = df['day'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    df = df[df['dt_day'] >= start_dt]
    df = df[df['dt_day'] <= end_dt]
    df['fill_time'] = df['time'].apply(lambda x: x.zfill(5))
    
    # plot average daily motion 
    dt = start_dt 
    dt_arr = []
    motion_arr = []
    while (dt <= end_dt):
        dt_arr.append(dt.strftime("%Y-%m-%d"))
        motion_arr.append(list(df[df['dt_day'] == dt]['motion'].values))
        dt += datetime.timedelta(days=1)

    plt.figure()
    plt.boxplot(motion_arr, labels=dt_arr)
    plt.xticks(rotation=45)
    plt.xlabel('date')
    plt.ylabel('average motion per 5 min interval')
    plt.savefig("plot.png", bbox_inches='tight')
        
    # Send the image
    with open('plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo)

    # plot average motion throughout the day 
    time_arr = []
    motion_arr = []
    curr_time = datetime.datetime(year=2023, month=1, day=1, hour=0, minute=0, second=0)
    end_time = datetime.datetime(year=2023, month=1, day=1, hour=23, minute=55, second=0)
    while (curr_time <= end_time): 
        time_arr.append(curr_time.strftime("%H:%M"))
        motion_arr.append(np.mean(df[df['fill_time'] == curr_time.strftime("%H:%M")]['motion'].values))
        curr_time += datetime.timedelta(minutes=5)

    plt.figure()
    plt.plot(time_arr, motion_arr)
    x_values = time_arr[::12] # show every 1 hour label 
    plt.xticks(x_values, rotation=45)
    plt.xlabel('time')
    plt.ylabel('average motion per 5 min interval')
    plt.savefig("plot.png", bbox_inches='tight')
        
    # Send the image
    with open('plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo)

    # output overall averages 
    msg = 'Overall Average Motion Per 5 min Interval: ' + str(np.mean(df['motion'].values))
    msg += '\nOverall Maximum Motion Per 5 min Interval: ' + str(np.max(df['motion'].values))
    bot.send_message(callback.message.chat.id, msg, reply_markup=markup)
    
@bot.callback_query_handler(func=lambda call: call.data == 'damp_stats')
def damp_stats(callback):
    # collecting november data 
    start_dt = datetime.datetime(2023, 11, 1)
    end_dt = datetime.datetime(2023, 11, 30)

    # data preparation 
    df = pd.read_csv(path)

    # filter to only contain november data 
    df['dt_day'] = df['day'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    df = df[df['dt_day'] >= start_dt]
    df = df[df['dt_day'] <= end_dt]
    df['fill_time'] = df['time'].apply(lambda x: x.zfill(5))

    # plot average daily dampness
    dt = start_dt 
    dt_arr = []
    damp_arr = []
    while (dt <= end_dt):
        dt_arr.append(dt.strftime("%Y-%m-%d"))
        damp_arr.append(list(df[df['dt_day'] == dt]['dampness'].values))
        dt += datetime.timedelta(days=1)

    plt.figure()
    plt.boxplot(damp_arr, labels=dt_arr)
    plt.xticks(rotation=45)
    plt.xlabel('date')
    plt.ylabel('dampness')
    plt.savefig("plot.png", bbox_inches='tight')
        
    # Send the image
    with open('plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo)

    # plot average dampness throughout the day 
    time_arr = []
    damp_arr = []
    curr_time = datetime.datetime(year=2023, month=1, day=1, hour=0, minute=0, second=0)
    end_time = datetime.datetime(year=2023, month=1, day=1, hour=23, minute=55, second=0)
    while (curr_time <= end_time): 
        time_arr.append(curr_time.strftime("%H:%M"))
        damp_arr.append(np.mean(df[df['fill_time'] == curr_time.strftime("%H:%M")]['dampness'].values))
        curr_time += datetime.timedelta(minutes=5)

    plt.figure()
    plt.plot(time_arr, damp_arr)
    x_values = time_arr[::12] # show every 1 hour label 
    plt.xticks(x_values, rotation=45)
    plt.xlabel('time')
    plt.ylabel('average dampness')
    plt.savefig("plot.png", bbox_inches='tight')
        
    # Send the image
    with open('plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo)

    # output overall average dampness 
    msg = 'Overall Average Dampness: ' + str(np.mean(df['dampness'].values))
    bot.send_message(callback.message.chat.id, msg, reply_markup=markup)

@bot.callback_query_handler(func=lambda call: call.data == 'trash_stats')
def trash_stats(callback):
    # collecting november data 
    start_dt = datetime.datetime(2023, 11, 1)
    end_dt = datetime.datetime(2023, 11, 30)
    
    # data preparation 
    df = pd.read_csv(path)
    
    # filter to only contain november data 
    df['dt_day'] = df['day'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    df = df[df['dt_day'] >= start_dt]
    df = df[df['dt_day'] <= end_dt]
    df['fill_time'] = df['time'].apply(lambda x: x.zfill(5))  
    
    # plot average daily trend of trashbin 
    time_arr = []
    trash_arr = []
    curr_time = datetime.datetime(year=2023, month=1, day=1, hour=0, minute=0, second=0)
    end_time = datetime.datetime(year=2023, month=1, day=1, hour=23, minute=55, second=0)
    while (curr_time <= end_time): 
        time_arr.append(curr_time.strftime("%H:%M"))
        trash_arr.append(np.mean(df[df['fill_time'] == curr_time.strftime("%H:%M")]['trash'].values)- 3)
        curr_time += datetime.timedelta(minutes=5)

    plt.figure()
    plt.plot(time_arr, trash_arr)
    x_values = time_arr[::12] # show every 1 hour label 
    plt.xticks(x_values, rotation=45)
    plt.xlabel('time')
    plt.ylabel('average trash capacity left/cm')
    plt.savefig("plot.png", bbox_inches='tight')
        
    # Send the image
    with open('plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo)

    # plot time when trash bin fills up 
    dt = start_dt 
    dt_arr = []
    trash_arr = []
    temp_dict = {}
    while (dt <= end_dt):
        dt_arr.append(dt.strftime("%Y-%m-%d"))
        temp_df = df[df['dt_day'] == dt]
        trash_arr.append(temp_df[temp_df['trash'] == 3].iloc[0]['fill_time'])
        dt += datetime.timedelta(days=1)

    temp_dict['date'] = dt_arr 
    temp_dict['time where trash bin is first filled'] = trash_arr
    temp_df = pd.DataFrame(temp_dict)
    
    msg = str(temp_df)
    bot.send_message(callback.message.chat.id, msg, reply_markup=markup)
    
@bot.callback_query_handler(func=lambda call: call.data == 'rating_stats')
def rating_stats(callback):
    # collecting november data 
    start_dt = datetime.datetime(2023, 11, 1)
    end_dt = datetime.datetime(2023, 11, 30)

    # data preparation 
    df = pd.read_csv(path)

    # filter to only contain november data 
    df['dt_day'] = df['day'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    df = df[df['dt_day'] >= start_dt]
    df = df[df['dt_day'] <= end_dt]
    df['fill_time'] = df['time'].apply(lambda x: x.zfill(5))
    
    # plot average daily ratings 
    dt = start_dt 
    dt_arr = []
    ratings_arr = []
    while (dt <= end_dt):
        dt_arr.append(dt.strftime("%Y-%m-%d"))
        ratings_arr.append(np.mean(df[df['dt_day'] == dt]['rating'].values))
        dt += datetime.timedelta(days=1)

    plt.figure()
    plt.plot(dt_arr, ratings_arr)
    plt.xticks(dt_arr, dt_arr, rotation=45)
    plt.xlabel('date')
    plt.ylabel('average rating')
    plt.savefig("plot.png", bbox_inches='tight')
        
    # Send the image
    with open('plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo)

    # plot average ratings throughout the day 
    time_arr = []
    ratings_arr = []
    curr_time = datetime.datetime(year=2023, month=1, day=1, hour=0, minute=0, second=0)
    end_time = datetime.datetime(year=2023, month=1, day=1, hour=23, minute=55, second=0)
    while (curr_time <= end_time): 
        time_arr.append(curr_time.strftime("%H:%M"))
        ratings_arr.append(np.mean(df[df['fill_time'] == curr_time.strftime("%H:%M")]['rating'].values))
        curr_time += datetime.timedelta(minutes=5)

    plt.figure()
    plt.plot(time_arr, ratings_arr)
    x_values = time_arr[::12] # show every 1 hour label 
    plt.xticks(x_values, rotation=45)
    plt.xlabel('time')
    plt.ylabel('average rating')
    plt.savefig("plot.png", bbox_inches='tight')
        
    # Send the image
    with open('plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo)

    # plot histogram of aggregated ratings 
    plt.figure()
    bin_edges = [1, 2, 3, 4, 5]  
    plt.hist(df['rating'].values, bins=bin_edges, color='c', alpha=0.7)
    plt.xlabel('ratings')
    plt.ylabel('count')
    plt.savefig("plot.png", bbox_inches='tight')
        
    # Send the image
    with open('plot.png', 'rb') as photo:
        bot.send_photo(callback.message.chat.id, photo)

    # output overall average rating 
    msg = 'Overall Average Rating for Month: ' + str(np.mean(df['rating'].values))
    bot.send_message(callback.message.chat.id, msg, reply_markup=markup)
    
bot.infinity_polling()

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but 